In [ ]:
import pandas as pd
import osgb
import gpxpy
import gpxpy.gpx
import lxml.etree as mod_etree

In [ ]:
df = pd.read_csv('coords.csv')
df.head(3)

In [ ]:
df['lat_long']=df['Grid Ref'].apply(osgb.parse_grid).apply(osgb.grid_to_ll)
df.head(3)

In [ ]:
gpx = gpxpy.gpx.GPX()
gpx.name = 'Chiltern Kanter Long Route'

# creates track and segments
track = gpxpy.gpx.GPXTrack()
gpx.tracks.append(track)

segment = gpxpy.gpx.GPXTrackSegment()
track.segments.append(segment)

for coords in df['lat_long']:
    segment.points.append(gpxpy.gpx.GPXTrackPoint(*coords))

In [ ]:
#definition of extension
namespace = '{opencpn}'

#create extension element
root = mod_etree.Element(f'{namespace}scale_min_max')
#mod_etree.SubElement(root, f'{namespace}UseScale')
root.attrib['UseScale'] = "true"
root.attrib['ScaleMin'] = "50000"
root.attrib['ScaleMax'] = "0"

In [ ]:
# uses itertuples and rows, rows need indexes instead of column names
for row in df.itertuples(index=False):
    waypoint = gpxpy.gpx.GPXWaypoint()
    waypoint.latitude = row[5][0]
    waypoint.longitude = row[5][1]
    waypoint.name = row[2]
    waypoint.description = row[3]
    waypoint.symbol = "Marks-Mooring-Float"
    waypoint.extensions.append(root)

    gpx.waypoints.append(waypoint)

In [ ]:
filename = 'chiltern'

with open(f'{filename}.gpx','w') as g_file:
    g_file.write(gpx.to_xml())